## 前処理

1. データ結合する2つのファイルをそれぞれ整形  
   - `cleaned_output.csv`（売上・年月・終了日データ）  
     - 不要な列の削除  
     - 日付の整形（終了日を `YYYY/MM/DD` に統一）  
   - `4gamer_output_correct.csv`（発売日・備考データ）  
     - 元の `4gamer_output.csv` に含まれていた欠損タイトルを手動で補完済み  
     - 必要な列の整形とクリーンアップを実施

2. タイトル列をキーにして、2つのファイルを内部結合
- 各タイトル各月の売上に欠損値有→売上平均で補完
- 補完後の前処理済ファイルを`merged_filled.csv`として保存

### 1.1. ライブラリとCSVの読み込み

In [ ]:
import pandas as pd
from decimal import Decimal

# ファイル読み込み（パスを必要に応じて調整）
file_path = "full_data_with_cleaned_titles.csv"
df = pd.read_csv(file_path)

### 1.2. タイトル整形と不要列削除

In [44]:
# タイトル列をクリーンタイトルで置き換え
df["タイトル"] = df["クリーンタイトル"]

# 不要な列（クリーンタイトル、関連ジャンル）を削除
df = df.drop(columns=["クリーンタイトル", "関連ジャンル"])

### 1.3. サービス終了日の整形

In [45]:
# サービス終了日を datetime に変換し、"YYYY/MM/DD" 形式に整形
df["サービス終了日"] = pd.to_datetime(df["サービス終了日"], errors='coerce').dt.strftime("%Y/%m/%d")

### 1.4.　売上予測を円に変換

In [46]:
# カンマやGを除去して、"億"/"万"単位を円に換算
def parse_sales(s):
    if pd.isna(s):
        return None
    s = str(s).replace("G", "").replace(",", "")
    if "億" in s:
        return float(Decimal(s.replace("億", "")) * Decimal("1e8"))
    elif "万" in s:
        return float(Decimal(s.replace("万", "")) * Decimal("1e4"))
    else:
        return None

### 1.5. 売上列の整理

In [47]:
# 売上予測（円）を作成して四捨五入
df["売上予測（円）"] = df["売上予測"].apply(parse_sales).round().astype("Int64")

# 元の「売上予測」列を削除
df = df.drop(columns=["売上予測"])

# 確認
df.head()

,タイトル,年月,サービス終了日,売上予測（円）
0,スーパーガンダムロワイヤル,2018/12,2021/07/29,248000000
1,スーパーガンダムロワイヤル,2018/11,2021/07/29,251000000
2,スーパーガンダムロワイヤル,2018/10,2021/07/29,263000000
3,スーパーガンダムロワイヤル,2018/09,2021/07/29,190000000
4,スーパーガンダムロワイヤル,2018/08,2021/07/29,198000000


### 1.6. 整形後データの保存

In [48]:
# 整形後データの保存
df.to_csv("cleaned_output.csv", index=False, encoding="utf-8-sig")

### 2.1. 発売日データの読み込み

In [49]:
# ファイルの読み込み
df_cleaned = pd.read_csv("cleaned_output.csv")  # 売上・年月・終了日など
df_4gamer = pd.read_csv("4gamer_output_corrected.csv")  # 発売日・備考などの情報

### 2.2. 2つのファイルを結合

In [50]:
#　結合前に"yyyy/mm"形式化
df_cleaned["年月"] = pd.to_datetime(df_cleaned["年月"],format="%Y/%m", errors="coerce").dt.strftime("%Y/%m")

# タイトルをキーにして内部結合
merged_df = pd.merge(df_cleaned, df_4gamer, on="タイトル", how="inner")

### 2.3. カラム名の整形と並べ替え

In [51]:
merged_df = merged_df.rename(columns={"発売日": "サービス開始日"})
columns_order = ["タイトル", "年月", "売上予測（円）", "サービス開始日", "サービス終了日", "備考"]
reordered_df = merged_df[columns_order]

### 2.4. 年月と日付をdatetime型に変換

In [52]:
reordered_df["年月"] = pd.to_datetime(reordered_df["年月"], format="%Y/%m", errors="coerce")
reordered_df["サービス開始日"] = pd.to_datetime(reordered_df["サービス開始日"], errors="coerce")
reordered_df["サービス終了日"] = pd.to_datetime(reordered_df["サービス終了日"], errors="coerce")

### 2.5.　タイトルごとの補完処理

In [58]:
def fill_with_title_mean(group):
    start = group["サービス開始日"].iloc[0].replace(day=1) # 月単位で補完するため1日で揃える
    end = group["サービス終了日"].iloc[0].replace(day=1)
    full_range = pd.date_range(start, end, freq="MS") # pb.date_range(s, e, frec): 日付の連続リスト作成
    
    group = group.set_index("年月").reindex(full_range) # group.set_index():指定列をインデックスとして設定/ .reindex():()で再配置し，元のdfに存在しない場合NaNで埋める
    group.index.name = "年月"
    
    for col in ["タイトル", "サービス開始日", "サービス終了日", "備考"]:
        group[col] = group[col].ffill().bfill()
        group = group.infer_objects(copy=False) # ffill():直前の値で補完、bfill():直後の値で補完

    group["売上予測（円）"] = pd.to_numeric(group["売上予測（円）"], errors="coerce") # pd.to_numeric():文字列や混合データを数値型に変換する関数
    title_mean = group["売上予測（円）"].mean()
    group["売上予測（円）"] = group["売上予測（円）"].fillna(title_mean)

    return group.reset_index() # reset_index():インデックスを普通の列に戻し，デフォルトの数値インデックスに置き換える
# 平均補完の実行
mean_filled_df = reordered_df.groupby("タイトル", group_keys=False).apply(fill_with_title_mean) 
# group_keys=False:グループキーを普通の列として残す　.apply():行または列に関数を一括して適用

C:\Users\zakky\AppData\Local\Temp\ipykernel_19472\3778886236.py:10: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  group[col] = group[col].ffill().bfill()
C:\Users\zakky\AppData\Local\Temp\ipykernel_19472\3778886236.py:19: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  mean_filled_df = reordered_df.groupby("タイトル", group_keys=False).apply(fill_with_title_mean)


### 2.6. 前処理データの保存

In [59]:
mean_filled_df.to_csv("merged_filled.csv", index=False, encoding="utf-8-sig")